In [ ]:
DATASET_NAME = "cardiffnlp/tweet_sentiment_multilingual"
MODEL_NAME = "bert-base-multilingual-cased"
CHECKPOINTS_PATH = '/content/drive/MyDrive/model_compression/model2.pt'

# Install requirements

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!git clone https://github.com/NeuroCrushers/model-compression-course

Cloning into 'model-compression-course'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 20 (delta 7), reused 18 (delta 5), pack-reused 0
Receiving objects: 100% (20/20), 29.44 KiB | 2.45 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [ ]:
%cd model-compression-course

/content/model-compression-course/model-compression-course


In [ ]:
!pip install -r requirements.txt

# Load model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load(CHECKPOINTS_PATH, map_location = device)

# Test model

In [ ]:
RANDOM_SEED = 42
BATCH_SIZE = 32
NUM_SHARDS = 1

In [ ]:
from inference import get_test_data

test_dataset, test_loader = get_test_data(dataset = DATASET_NAME,
                                          random_seed = RANDOM_SEED,
                                          num_shards = NUM_SHARDS,
                                          batch_size = BATCH_SIZE)

In [ ]:
import inference
from importlib import reload
reload(inference)

<module 'inference' from '/content/model-compression-course/model-compression-course/inference.py'>

In [ ]:
from inference import eval_all_langs, scores_to_df, measure_size_mb, measure_inference_time

In [ ]:
LANGUAGES = ['arabic', 'english', 'french', 'german', 'hindi', 'italian', 'portuguese', 'spanish']
device = 'cuda' if torch.cuda.is_available() else 'cpu'
scores = eval_all_langs(dataset = DATASET_NAME,
                        model = model, tokenizer = tokenizer,
                        device = device, languages = LANGUAGES)

arabic


  0%|          | 0/28 [00:00<?, ?it/s]

F1 for batch 27: 0.5222222222222223
Total F1 score: 0.49
english


  0%|          | 0/28 [00:00<?, ?it/s]

F1 for batch 27: 0.5222222222222223
Total F1 score: 0.51
french


...

F1 for batch 27: 0.5222222222222221
Total F1 score: 0.51
portuguese


  0%|          | 0/28 [00:00<?, ?it/s]

F1 for batch 27: 0.4000000000000001
Total F1 score: 0.55
spanish


  0%|          | 0/28 [00:00<?, ?it/s]

F1 for batch 27: 0.8222222222222223
Total F1 score: 0.54


In [ ]:
scores_to_df(scores)

,f1
arabic,0.491842
english,0.507352
french,0.586372
german,0.585325
hindi,0.426980
italian,0.511367
portuguese,0.552181
spanish,0.541485
mean,0.525363


In [ ]:
measure_size_mb(model)

Model size: 678.474MB


In [ ]:
measure_inference_time(model = model,
                       tokenizer = tokenizer,
                       device = device,
                       loader = test_loader)

  0%|          | 0/218 [00:00<?, ?it/s]

Avg inference time: 1437.08065533638
